Our goal here is to use BankChurners.csv file and identify the factors that leads to customer exiting or churning. We need to identify the set of target customers which could be contacted to prevent them from churning. Also used a basic algorithm (Random forest classifier) is used here for predicting churning.

In [ ]:
# Libraries
import numpy as np
import pandas as pd
# visualization lib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandasql as psql
import plotly.express as ex
import plotly.graph_objs as go
#  
from pprint import pprint
from IPython.display import display 
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import confusion_matrix,plot_confusion_matrix,precision_recall_fscore_support
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score
from sklearn.model_selection import RandomizedSearchCV

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))      
        
pd.set_option("display.max_columns", 100)
pd.set_option('display.width', 1000)
path = '/kaggle/input/credit-card-customers/BankChurners.csv'
df = pd.read_csv(path)
#-------------data processing  - drop Clientnum and last 2 columns ----------------------
df.drop(df.columns[[0,-1,-2]].values,axis=1,inplace=True)
# df.head()
# df
df.shape

In [ ]:
# df['Attrition_Flag'].value_counts()
#df.dtypes
#df.columns.values.tolist()
# df.describe()

#Data cleaning
df.isnull().sum()

The dataset is clean !

In [ ]:
ex.pie(df,names='Attrition_Flag',title='Existing and Attirated Customers proportions')

In [ ]:
# attrition relation with variables
fig, axarr = plt.subplots(2, 2, figsize=(20, 12))
sns.countplot(x='Customer_Age', hue = 'Attrition_Flag',data = df, ax=axarr[0][0])
sns.countplot(x='Gender', hue = 'Attrition_Flag',data = df, ax=axarr[0][1])
sns.countplot(x='Dependent_count', hue = 'Attrition_Flag',data = df, ax=axarr[1][0])
sns.countplot(x='Education_Level', hue = 'Attrition_Flag',data = df, ax=axarr[1][1])

We note the following: Majority of customers who churn are between 40 to 50 years of age. Churning is high for education level graduate and high school. The proportion of female customers churning is also greater than that of male customers.

However the pattern is consistent with the existing customers for the respective section of above demographic variables.

In [ ]:
fig, axarr = plt.subplots(2, 2, figsize=(20, 12))
sns.countplot(x='Marital_Status', hue = 'Attrition_Flag',data = df, ax=axarr[0][0])
sns.countplot(x='Income_Category', hue = 'Attrition_Flag',data = df, ax=axarr[0][1])
sns.countplot(x='Total_Relationship_Count', hue = 'Attrition_Flag',data = df, ax=axarr[1][0])
sns.countplot(x='Months_on_book', hue = 'Attrition_Flag',data = df, ax=axarr[1][1])

Customers in income categeory of less than $40k churn more than othe income categeory Interestingly most of the customers who churned have 36 months of relationship with the bank. Also the more the relationship counts are the more customer is less likely to churn.

However the pattern is consistent with the existing customers for the respective section of above demographic variables.



In [ ]:
sns.countplot(x='Months_Inactive_12_mon', hue = 'Attrition_Flag',data = df)

The churning customers show more inactivity in last 3 months.

In [ ]:
sns.countplot(x='Contacts_Count_12_mon', hue = 'Attrition_Flag',data = df)

This field shows a weak negative corellation with the Attirition of the customer. The pattern looks very consistant and the relationship can be used for predictiong the future churning customers The churned customers showed more contacts in last months as compared to the existing customers.

In [ ]:
sns.boxplot(y='Credit_Limit',x = 'Attrition_Flag', hue = 'Attrition_Flag',data = df)

There is a slight difference of credit limit distribution between existing and churned customers.The limit for churned customer is on relatively lower side as compared to the credit limit of existing customers but not very significant.

In [ ]:
sns.boxplot(y='Total_Revolving_Bal',x = 'Attrition_Flag', hue = 'Attrition_Flag',data = df)

The revolving balance of the attrited customers is lower than non churned customers. Total_Revolving_Bal shows a strong positive corellation with the Attirition_flag of the customer and is very consistant can be used for predictiong the future churning customers

In [ ]:
sns.boxplot(y='Avg_Open_To_Buy',x = 'Attrition_Flag', hue = 'Attrition_Flag',data = df)

In [ ]:
sns.boxplot(y='Total_Amt_Chng_Q4_Q1',x = 'Attrition_Flag', hue = 'Attrition_Flag',data = df)


No clear difference here


In [ ]:
sns.boxplot(y='Total_Trans_Amt',x = 'Attrition_Flag', hue = 'Attrition_Flag',data = df)

As clear Total trans amount is lower side for attirated customers than existing customers.

In [ ]:
sns.boxplot(y='Total_Ct_Chng_Q4_Q1',x = 'Attrition_Flag', hue = 'Attrition_Flag',data = df)

The change ct from Q4 to Q1 above is lower for churned customers.

In [ ]:
sns.boxplot(y='Avg_Utilization_Ratio',x = 'Attrition_Flag', hue = 'Attrition_Flag',data = df)


Average utilization ration is less for churned customers as compared to the existing customers which could be helpful in finding potential churn customers.

In [ ]:
df2 = pd.read_csv(path)
df2.drop(df2.columns[[0,-1,-2]].values,axis=1,inplace=True)
df2['Attrition_Flag'].replace({'Existing Customer':0, 'Attrited Customer':1},inplace=True)

In [ ]:
# correlation between variables on each axis
plt.figure(figsize = (12,12))
sns.heatmap(df2.corr(), annot = True, cmap = 'RdYlGn')

To summarize : We see mild linear trend between Months contact count and attiration flag as well as Months inactive.

Few negative linear trends between attiration flag and : 1 total trans count. 2 total revolving balance.
3 total ct chnge Q4 to Q1 is observed. 4 total average utilization ratio.

Apart from trends with Attiration we also observe following mutual trends among diff variables : There is a negative linear trend between credit limit and Avg utilization ratio. similarly as between credit limit and Avg open to buy. Their is a positive linear trend between Avg UTIL ratio and Total Revolving balance.(Revolving balance and attiration -ve)

Already discussed in earlier charts that no. of months a customer was active and his last contacts could help us decide if this customer may churn and can be made active by offers etc.

In [ ]:
df2.describe()

In [ ]:
# Taking all the strong linear trends from above heatmap in a sql queries for their mean and std values
# would mostly return attiration flag with 1 
df2 = psql.sqldf("select  * from df2 where Contacts_Count_12_mon > '2.4' or Total_Revolving_Bal < '814' or Total_Trans_Ct <'23'or Total_Ct_Chng_Q4_Q1 < '0.23'")
df2

In [ ]:
df2 = psql.sqldf("select count(*) from df2 where Attrition_Flag = '1'")
df2
#( we have 1627 rows of attiration cust and out of them we get 1462 records with attiration 1 by filtering 
# on these columns on df2) 
# ( filter is Contacts_Count_12_mon > '2.4' or Total_Revolving_Bal < '814' or 
# Total_Trans_Ct <'23'or Total_Ct_Chng_Q4_Q1 < '0.23')

In [ ]:
# Data prep for random forest
# convert bool and object to category 
cat_types = ['bool','object','category']
df[df.select_dtypes(cat_types).columns] = df.select_dtypes(cat_types).apply(lambda x: x.astype('category'))
df.info()

In [ ]:
# Map Attrited Customer = 1 and Existing Customer = 0
codes = {'Existing Customer':0, 'Attrited Customer':1}
df['Attrition_Flag'] = df['Attrition_Flag'].map(codes)

Y = df['Attrition_Flag']
X = df.drop('Attrition_Flag',errors='ignore',axis=1)

# print the categories for each columns 
for col in X.select_dtypes('category').columns.to_list():
    print(col + ': '+ str(X[col].cat.categories.to_list()))

# Use One Hot Eoncoding on each catagorical independent variables 
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = X.select_dtypes('category').columns.to_list()
for feature in features_to_encode:
    X = encode_and_bind(X, feature)
    
X.info()

In [ ]:
#train-test stratified split 
X_train80, X_test20, Y_train80, Y_test20 = train_test_split(X, Y, test_size=0.2, random_state = 0, shuffle= True,stratify = Y)
# 
print(Y_train80.value_counts())
print(Y_test20.value_counts())

In [ ]:
# Initial fit using RandomForestClassifier
RFC = RandomForestClassifier(random_state = 0)
RFC.fit(X_train80,Y_train80)
print("Accuracy: %.2f%%" % ((RFC.score(X_test20,Y_test20))*100.0))


In [ ]:
# Confusion Matrix for RFC
np.set_printoptions(precision=2)
# Plot Both Confusion Matrix, without normalization and with normalization 
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(RFC, X_test20, Y_test20,
                                 display_labels=RFC.classes_,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)
plt.show()

In [ ]:
# predictions using RFC model given X_test20 data
Y_pred20 = RFC.predict(X_test20)
# precision, recall, f1-score
precision_recall_fscore_support(Y_test20, Y_pred20, average='binary',pos_label=1,beta = 1)
# classification_report for Attrited Customer  
print(classification_report(Y_test20,Y_pred20))
print("Accuracy: %.2f%%" % (accuracy_score(Y_test20, Y_pred20)*100.0))
print("Recall: %.2f%%" % ((recall_score(Y_test20,Y_pred20))*100.0))

Accurarcy is 95.66% and recall which is 76.00%.This could be further improved by trying other machine learning algorithms.